# Homework 8

## FINM 36700 - 2024

### Group C 44
* Andy Andikko: andyandikko@uchicago.edu
* Harrison Zhang: harrisonzhang@uchicago.edu
* Matheus Raka Pradnyatama matheusraka@uchicago.edu

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import jinja2

from scipy.stats import norm
from functools import partial
from typing import Union, List
from sklearn.linear_model import LinearRegression

pd.options.display.float_format = "{:,.4f}".format
plt.style.use('ggplot')

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
def summary_statistics_annualized(returns, annual_factor = 12): #returns index is date
    """This functions returns the summary statistics for the input total/excess returns passed
    into the function"""
    summary_statistics = pd.DataFrame(index=returns.columns)
    summary_statistics['Mean'] = returns.mean() * annual_factor
    summary_statistics['Volatility'] = returns.std() * np.sqrt(annual_factor)
    summary_statistics['Sharpe Ratio'] = (returns.mean() / returns.std()) * np.sqrt(annual_factor)
    #summary_statistics['Min'] = returns.min()
    #summary_statistics['Max'] = returns.max()
    summary_statistics['Skewness'] = returns.skew()
    summary_statistics['Excess Kurtosis'] = returns.kurtosis()
    summary_statistics['VaR (0.05)'] = returns.quantile(.05, axis = 0)
    #summary_statistics['CVaR (0.05)'] = returns[returns <= returns.quantile(.05, axis = 0)].mean()
    return summary_statistics

In [3]:
def time_series_regression(portfolio, factors, multiple_factors = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] * 12
        if multiple_factors:
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]
            ff_report.loc[portf, factors.columns[1]+ ' beta'] = res.params[2] 
            ff_report.loc[portf, factors.columns[2]+ ' beta'] = res.params[3]
        else:
            ff_report.loc[portf, factors.name + ' beta'] = res.params[1]

            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

In [4]:
def calc_return_metrics(data, as_df=False, adj=12):
    """
    Calculate return metrics for a DataFrame of assets.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        as_df (bool, optional): Return a DF or a dict. Defaults to False (return a dict).
        adj (int, optional): Annualization. Defaults to 12.

    Returns:
        Union[dict, DataFrame]: Dict or DataFrame of return metrics.
    """
    summary = dict()
    summary["Annualized Return"] = data.mean() * adj
    summary["Annualized Volatility"] = data.std() * np.sqrt(adj)
    summary["Annualized Sharpe Ratio"] = (
        summary["Annualized Return"] / summary["Annualized Volatility"]
    )
    summary["Annualized Sortino Ratio"] = summary["Annualized Return"] / (
        data[data < 0].std() * np.sqrt(adj)
    )
    return pd.DataFrame(summary, index=data.columns) if as_df else summary

def calc_risk_metrics(data, as_df=False, var=0.05):
    """
    Calculate risk metrics for a DataFrame of assets.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        as_df (bool, optional): Return a DF or a dict. Defaults to False.
        adj (int, optional): Annualizatin. Defaults to 12.
        var (float, optional): VaR level. Defaults to 0.05.

    Returns:
        Union[dict, DataFrame]: Dict or DataFrame of risk metrics.
    """
    summary = dict()
    summary["Skewness"] = data.skew()
    summary["Excess Kurtosis"] = data.kurtosis()
    summary[f"VaR ({var})"] = data.quantile(var, axis=0)
    summary[f"CVaR ({var})"] = data[data <= data.quantile(var, axis=0)].mean()
    summary["Min"] = data.min()
    summary["Max"] = data.max()

    wealth_index = 1000 * (1 + data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary["Max Drawdown"] = drawdowns.min()

    summary["Bottom"] = drawdowns.idxmin()
    summary["Peak"] = previous_peaks.idxmax()

    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][: drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin() :]]).T
        recovery_date.append(
            recovery_wealth[recovery_wealth[col] >= prev_max].index.min()
        )
    summary["Recovery"] = ["-" if pd.isnull(i) else i for i in recovery_date]

    summary["Duration (days)"] = [
        (i - j).days if i != "-" else "-"
        for i, j in zip(summary["Recovery"], summary["Bottom"])
    ]

    return pd.DataFrame(summary, index=data.columns) if as_df else summary

def calc_performance_metrics(data, adj=12, var=0.05):
    """
    Aggregating function for calculating performance metrics. Returns both
    risk and performance metrics.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        adj (int, optional): Annualization. Defaults to 12.
        var (float, optional): VaR level. Defaults to 0.05.

    Returns:
        DataFrame: DataFrame of performance metrics.
    """
    summary = {
        **calc_return_metrics(data=data, adj=adj),
        **calc_risk_metrics(data=data, var=var),
    }
    summary["Calmar Ratio"] = summary["Annualized Return"] / abs(
        summary["Max Drawdown"]
    )
    return pd.DataFrame(summary, index=data.columns)

In [5]:
def calc_multivariate_regression(y, X, intercept=True, adj=12):
    """
    Calculate a multivariate regression of y on X. Adds useful metrics such
    as the Information Ratio and Tracking Error. Note that we can't calculate
    Treynor Ratio or Downside Beta here.

    Args:
        y : target variable
        X : independent variables
        intercept (bool, optional): Defaults to True.
        adj (int, optional): Annualization factor. Defaults to 12.

    Returns:
        DataFrame: Summary of regression results
    """
    if intercept:
        X = sm.add_constant(X)

    model = sm.OLS(y, X, missing="drop")
    results = model.fit()
    summary = dict()

    inter = results.params.iloc[0] if intercept else 0
    betas = results.params.iloc[1:] if intercept else results.params

    summary["Alpha"] = inter * adj
    summary["R-Squared"] = results.rsquared

    X_cols = X.columns[1:] if intercept else X.columns

    for i, col in enumerate(X_cols):
        summary[f"{col} Beta"] = betas[i]
    
    #summary['Sortino Ratio'] = y.mean() / y[y< 0].std() * np.sqrt(adj)
    #summary["Information Ratio"] = (inter / results.resid.std()) * np.sqrt(adj)
    #summary["Tracking Error"] = results.resid.std() * np.sqrt(adj)
   
    if isinstance(y, pd.Series):
        return pd.DataFrame(summary, index=[y.name])
    else:
        return pd.DataFrame(summary, index=y.columns)

## 1.1 

Securities traded:

LTCM tried to trade on market mispricing and arbitrage, Relative Value and Convergence trades. They go long-short on these arbitrages. Use leverage to trade bigger principal on these small mispricings and try to hedge out their positions via their long-short trades. They primarily used derivatives, in the form of swaps to achieve these positions.

LTCM was also heavily involved income and credit, and they also have sizeable positions in equities. In all these asset classes, they trade a large number of securities, across global markets.

Trading frequency:

LTCM's trading frequency varied according to their strategies. Their largest investment in the form of convergence trades had a long term trading horizon and frequency (weeks or months). They are not trying to arbitrage intraday movements and nor do they make long-term directional bets. 

Skewness: 

They are picking up pennies in front of a bulldozer. So, many small wins. They seek small positive returns using leverage and do not bet significantly on any specific events. Have lower skewness than SPY. However, they are susceptible to extreme market events (it was the Russian currency crisis that brought them down).

Forecasting: 

Build models to find mispricing and the reason behind the mispricing. Then forecast their P&L on these trades. Their forecast is not better because of better mathematical model (the convergence trade/relative value theory is not the edge), it is their knowledge of the market.


## 1.2

1. Efficient financing: Their edge was on financing and funding, along with their proprietary trading and modelling capabilities.
2. Fund Size: They had a larger AUM, meaning they could lever at favorable rates
3. Collateralization: Better collateralize these positions. (pay lower haircuts)
4. Long-term Horizon: Long term commitment of capital from investors as well as availability of credit line
5. Liquidity and Hedging: LTCM has in place many mechanisms to ensure liquidity. They also avoid taking too much default risk or explicit directional bets. 

## 1.3

Collateral haircuts:

The haircuts go up in a market disruption event leading to unfavorable collateral terms for LTCM in terms of funding a spread trade. For most trades, LTCM obtains 100% financing on a fully collateralized basis. Furthermore, LTCM stress tests the haircuts across its asset classes.

Repo maturity:

In an adverse situation, where their credit risk goes up, they wont be able to secure these longer term repos which were favorable to their trades. LTCM goes against the norm by entering into relatively long-maturity repo. While much of it is overnight, LTCM uses contracts that typically have maturity of 6-12 months. Furthermore, LTCM manages their aggregate repo maturity.

Equity redemption:

If in a convergence trade, the two securities, before converging, diverge a lot, LTCM are facing redemption risk from their investors at a time where the Margin calls need them to further finance their strategies. Equity Redemption at a unfavorable time also leads LTCM to unwind their positions at unfavorable rates leading to further losses of capital. The firm is highly levered, so equity funding risk is especially important. LTCM restricts redemptions of equity year by year. The restriction is particularly strong in that unredeemed money is re-locked.

Loan access:

Loan access can be tough to come by in times of a crisis, leading to a further decline in the fund's performance. For debt funding, LTCM negotiated a revolving loan that has no Material Adverse Change clause. Thus, the availability of debt funding is not so highly correlated with fund performance. 

## 1.4

LTCM required counterparties to maintain the collateral balance via a 'two-way mark-to-market process on a daily basis. Thus, the cash flow coming in from the counterparties mark-to-market would fund LTCM's outflow for the mark-to-market call on their offsetting position.

LTCM also estimated theoretical worst case haircuts it would face in adverse market situations. Forecasting these worst case liquidity, LTCM was able to better structure its financing so as not to liquidate its positions solely due to these adverse market events.

LTCM attempts to account for liquidity risk quantitatively by adjusting security correlations. For short-term horizons, LTCM assumes positive correlation between all trade categories. Even if their net exposure to a strategy flips sides, they still assume positive correlation to the new net position.

## 1.5

Currently since there were no extreme market events, leverage risk is not a concern, but still a potential threat for LTCM. Given the size of their commited capital and fewer opportunites for the excess capital to enhance LTCM's return, they are considering returning some of the investments made, which would reduce the leverage.

Note: the amount of "true" leverage is also frequently misreported. The reason being that SEC filings require the reporting of the gross notional exposure, not the net exposure! As an example, consider [this article](https://www.cnn.com/2023/08/15/investing/michael-burry-stock-market-crash/index.html). It claims that Michael Burry "bet" $1.6 billion on a market crash. In reality, his exposure is $1.6 billion; he achieved this by buying put options for much, much, cheaper (potentially as low as ~$10m in premiums; capping his losses at $10m).

## 1.6

About a year after the time of the case, the fund loses most of its value due to non-converging trades. So clearly there is some risk!

Positions are subject to liquidity risk. If market liquidity dries up or the markets become segmented, the divergent spreads can persist for a long time. This indeed happens later to LTCM. The trades that get them in trouble ultimately pay off, but not before LTCM blew up. LTCM believed it can exit these convergence trades if they become too unprofitable. However, a stop-loss order is not the same as a put option. If the price jumps discontinuously through the stop-loss, then it is ineffective.

Or a market may be paralyzed/illiquid when trying to execute the stop-loss. A put option does not need to worry about price impact, whereas a stop-loss does. Finally, a stop-loss ensures that an investor sells as soon as a security price hits a worst-case scenario, ensuring unfavorable market timing.

# 2. LTCM Risk Decomposition

In [6]:
path1 = '/Users/matheus/Desktop/finm-portfolio-2024/data/ltcm_exhibits_data.xlsx'
path2 = '/Users/matheus/Desktop/finm-portfolio-2024/data/gmo_analysis_data.xlsx'

# Read ltcm_exhibits_data.xlsx and get sheet Exhibit 2
ltcm = pd.read_excel(path1, sheet_name='Exhibit 2', skiprows=2, index_col=0, parse_dates=[0]).iloc[:-4, :]
ltcm.index.name = 'Date'

# Get SPY returns from returns (total) in gmo_analysis_data.xlsx
spy = pd.read_excel(path2, sheet_name='total returns', index_col=0, parse_dates=[0])[['SPY']]
us3m = pd.read_excel(path2, sheet_name='risk-free rate', index_col=0, parse_dates=[0])[['TBill 3M']]

# Strip the time portion if present and ensure proper datetime conversion
# pd.tseries.offsets.MonthEnd(0) is to align the dates to the end of their respective months.
ltcm.index = pd.to_datetime(ltcm.index.astype(str).str.split().str[0]) + pd.tseries.offsets.MonthEnd(0)

# Grab only gross and net performance and drop na values
ltcm = ltcm[['Net Monthly Performanceb', 'Gross Monthly Performancea']].dropna()
# rename columns
ltcm = ltcm.rename(columns={'Net Monthly Performanceb': 'LTCM Net', 'Gross Monthly Performancea': 'LTCM Gross'})

ltcm.head(2)

,LTCM Net,LTCM Gross
Date,,
1994-03-31,-0.0130,-0.0110
1994-04-30,0.0080,0.0140


In [7]:
# Join on month and year of the index
df = ltcm.join(spy, how='inner').join(us3m, how='inner')
df.head(2)

,LTCM Net,LTCM Gross,SPY,TBill 3M
1994-03-31,-0.0130,-0.0110,-0.0419,0.0368
1994-04-30,0.0080,0.0140,0.0112,0.0414


In [8]:
# Convert to EXCESS returns
df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']] = df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']].subtract(df['TBill 3M'], axis=0)
df.head(2)

,LTCM Net,LTCM Gross,SPY,TBill 3M
1994-03-31,-0.0498,-0.0478,-0.0787,0.0368
1994-04-30,-0.0334,-0.0274,-0.0302,0.0414


## 2.1

### a) and b)

In [9]:
# obtain the performance metrics of LTCM excess returns and SPY's
summary1 = summary_statistics_annualized(df[['LTCM Net', 'LTCM Gross', 'SPY']], annual_factor = 12).T
summary1

,LTCM Net,LTCM Gross,SPY
Mean,-0.3634,-0.2684,-0.3808
Volatility,0.1079,0.1330,0.1053
Sharpe Ratio,-3.3673,-2.0179,-3.6172
Skewness,-0.7762,-0.2197,-0.3929
Excess Kurtosis,3.9278,2.0895,-0.1349
VaR (0.05),-0.0641,-0.0675,-0.0904


## c)
LTCM has a slightly a higher mean (less negative) but higher vol compared to SPY. Since its mean is slightly better than SPY's, its risk-adjusted return is also better. This is true for both net and gross performance.

Both LTCM Net and Gross have negative skew, indicating a higher probability of extreme losses compared to gains. SPY also have a negative skew, but less than LTCM Net. 

LTCM Net and Gross both have higher excess kurtosis than SPY. This high kurtosis of LTCM's performance means that there is a greater probability of extreme events (both extreme positive and negative events), which increases tail risk.

Both LTCM Net and Gross have less extreme VaR compared to SPY. This means that LTCM is less risky if we are comparing only the VaR.

In conclusion, LTCM does have significant tail risk, indicated by the high excess kurtosis and a very negative skewness. Compared to SPY, LTCM is more susceptible to swings in market events and has a higher volatility. There isn't much difference between Gross and Net Performance.

## 2.2

a)

In [10]:
y = df[['LTCM Net']]
X = df[['SPY']]
calc_multivariate_regression(y, X, intercept=True, adj=12)

,Alpha,R-Squared,SPY Beta
LTCM Net,-0.3378,0.0043,0.0670


b) Definitely not. Low beta and low R-squared means that the fund that the fund is not tracking the market and is in no way a closet indexer. This is likely because LTCM is primarily trading fixed income, and most of their trades are pairs/arbitrage trades, meaning that by definition, they don't have exposure to the market.

c) No, since the alpha of the LTCM Net Excess Return is negative, it means that LTCM doesn't deliver excess returns beyond the risk premium we expect from market exposure.

## 2.3
### a)

In [11]:
df[['SPY2']] = df[['SPY']] ** 2
df.head(2)

,LTCM Net,LTCM Gross,SPY,TBill 3M,SPY2
1994-03-31,-0.0498,-0.0478,-0.0787,0.0368,0.0062
1994-04-30,-0.0334,-0.0274,-0.0302,0.0414,0.0009


In [12]:
y = df[['LTCM Net']]
X = df[['SPY', 'SPY2']]
calc_multivariate_regression(y, X, intercept=True, adj=12)

,Alpha,R-Squared,SPY Beta,SPY2 Beta
LTCM Net,-0.3440,0.0059,-0.0152,-1.0986


### b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

The quadratic market factor didn't significantly increase the overall LTCM variation explained by the market, but slightly increase it. We can observe this from the higher R-squared of 0.59% of this model, compared to the previously 0.43% if we don't use a quadratic market factor. The huge negative beta on SPY squared is a feature of the factor. The monthly returns are small, so the squared returns are even smaller, thus the beta has to be larger in magnitude to fit these small returns properly.

### c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

The beta to SPY can be thought of as the delta of the market options and the beta to SPY squared as the gamma of the market options. Since the gamma of an option is always positive, the negative beta of SPY squared indicates that LTCM is short the positive gamma, which means it is short the market options. 

### d) Should we describe LTCM as being positively or negatively exposed to market volatility?

Beta measures how volatile an asset is to the market return. With a negative beta for SPY squared, big market movements would lead to big underperformance of LTCM. Thus, LTCM seems to be negatively exposed to market volatility. LTCM will underperform if there are big deviations in the market, but will not be impacted as much by small deviations in the market.

## 2.4 
### a)

In [13]:
k1 = 0.03
k2 = -0.03
df[['SPY_Call']] = np.maximum(df[['SPY']] - k1, 0)
df[['SPY_Put']] = np.maximum(k2 - df[['SPY']], 0)

y = df[['LTCM Net']]
X = df[['SPY', 'SPY_Call', 'SPY_Put']]
calc_multivariate_regression(y, X, intercept=True, adj=12)

,Alpha,R-Squared,SPY Beta,SPY_Call Beta,SPY_Put Beta
LTCM Net,-0.3549,0.0089,-0.0668,0.0000,-0.2268


### (b) Is LTCM long or short the call-like factor? And the put-like factor?
LTCM is neutral for the call-like factor (zero beta), and short the put-like factor (negative beta).

### (c) Which factor moves LTCM more, the call-like factor, or the put-like factor?
The put-like factor moves LTCM more because it has a bigger beta magnitude compared to the call-like factor. A big movement in the put-like factor will lead to an underperformance of LTCM. The put-like behavior dominates.

### (d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?
It seems that the volatility exposure comes more from being short the market downside (the put-like factor) based on this current regression. But we cannot conclude this based on this regression alone. The answer to this question depends on the strikes of the options. 

# 3 The FX Carry Trade

In [27]:
# Risk-Free Rates
path3 = '/Users/matheus/Desktop/finm-portfolio-2024/data/fx_rf_data.xlsx'
risk_free_rates = pd.read_excel(path3, sheet_name='risk-free rates')
risk_free_rates.index = risk_free_rates['date']
risk_free_rates = risk_free_rates.drop(['date'],axis=1) # drop the date column

for col in risk_free_rates.columns:
    # The risk free rates are annualized numbers
    # Make them into daily numbers
    risk_free_rates[col] = risk_free_rates[col] / 252
    # Add the log for each column
    risk_free_rates['log_'+col] = np.log(1+risk_free_rates[col])

risk_free_rates.head(3)

,USD,JPY,EUR,GBP,MXN,CHF,log_USD,log_JPY,log_EUR,log_GBP,log_MXN,log_CHF
date,,,,,,,,,,,,
2008-01-01,0.0002,0.0000,0.0002,0.0002,0.0003,0.0001,0.0002,0.0000,0.0002,0.0002,0.0003,0.0001
2008-01-02,0.0002,0.0000,0.0001,0.0002,0.0003,0.0001,0.0002,0.0000,0.0001,0.0002,0.0003,0.0001
2008-01-03,0.0002,0.0000,0.0002,0.0002,0.0003,0.0001,0.0002,0.0000,0.0002,0.0002,0.0003,0.0001


In [28]:
# Spot FX Rates
fx_rates = pd.read_excel(path3, sheet_name='exchange rates')
fx_rates.index = fx_rates['date']
fx_rates = fx_rates.drop(['date'],axis=1) # drop the date column

for col in fx_rates.columns:
    fx_rates[col] = fx_rates[col]
    # Add the log for each column
    fx_rates['log_'+col] = np.log(fx_rates[col])

fx_rates.head(3)

,JPY,EUR,GBP,MXN,CHF,log_JPY,log_EUR,log_GBP,log_MXN,log_CHF
date,,,,,,,,,,
2008-01-01,0.0090,1.4592,1.9864,0.0918,0.8824,-4.7153,0.3779,0.6863,-2.3878,-0.1251
2008-01-02,0.0091,1.4715,1.9808,0.0916,0.8933,-4.6974,0.3863,0.6835,-2.3901,-0.1128
2008-01-03,0.0091,1.4750,1.9710,0.0919,0.9002,-4.6942,0.3887,0.6785,-2.3874,-0.1051


## 3.1 The Static Carry Trade

In [ ]:
# fx_rates['log_JPY'].shift(-1) is at time t+1
# fx_rates['log_JPY'] is at time t
# risk_free_rates['log_JPY'] is at time t --> t+1
# ret = spot_{t+1} - spot_{t} + rf_{foreign,t,t+1} - rf_{USD,t,t+1}


fx_hldg_excess_ret_JPY = fx_rates['log_JPY'].shift(-1) - fx_rates['log_JPY'] + risk_free_rates['log_JPY'] - risk_free_rates['log_USD']
fx_hldg_excess_ret_JPY = fx_hldg_excess_ret_JPY.to_frame('JPY')

fx_hldg_excess_ret_EUR = fx_rates['log_EUR'].shift(-1) - fx_rates['log_EUR'] + risk_free_rates['log_EUR'] - risk_free_rates['log_USD']
fx_hldg_excess_ret_EUR = fx_hldg_excess_ret_EUR.to_frame('EUR')

fx_hldg_excess_ret_GBP = fx_rates['log_GBP'].shift(-1) - fx_rates['log_GBP'] + risk_free_rates['log_GBP'] - risk_free_rates['log_USD']
fx_hldg_excess_ret_GBP = fx_hldg_excess_ret_GBP.to_frame('GBP')

fx_hldg_excess_ret_MXN = fx_rates['log_MXN'].shift(-1) - fx_rates['log_MXN'] + risk_free_rates['log_MXN'] - risk_free_rates['log_USD']
fx_hldg_excess_ret_MXN = fx_hldg_excess_ret_MXN.to_frame('MXN')

fx_hldg_excess_ret_CHF = fx_rates['log_CHF'].shift(-1) - fx_rates['log_CHF'] + risk_free_rates['log_CHF'] - risk_free_rates['log_USD']
fx_hldg_excess_ret_CHF = fx_hldg_excess_ret_CHF.to_frame('CHF')

fx_combined = pd.concat([fx_hldg_excess_ret_JPY, fx_hldg_excess_ret_EUR, fx_hldg_excess_ret_GBP, fx_hldg_excess_ret_MXN, fx_hldg_excess_ret_CHF], axis=1)
fx_combined = fx_combined.dropna() # drop NA rows
fx_combined.head()

,JPY,EUR,GBP,MXN,CHF
date,,,,,
2008-01-01,0.0178,0.0084,-0.0028,-0.0022,0.0122
2008-01-02,0.0030,0.0024,-0.0049,0.0029,0.0076
2008-01-03,0.0065,-0.0005,0.0016,-0.0035,0.0021
2008-01-04,-0.0056,-0.0032,-0.0018,0.0025,-0.0073
2008-01-07,0.0025,0.0007,0.0015,-0.0026,0.0035


In [32]:
fx_summary = summary_statistics_annualized(fx_combined, annual_factor = 252) # The data was in days
fx_summary[['Mean', 'Volatility', 'Sharpe Ratio']]

,Mean,Volatility,Sharpe Ratio
JPY,-0.0292,0.0991,-0.2942
EUR,-0.0231,0.0900,-0.2568
GBP,-0.0243,0.0957,-0.2544
MXN,0.0131,0.1300,0.1007
CHF,0.0031,0.1069,0.0292


JPY, EUR, and GBP all have negative returns. MXN and CHF have had positive returns. Something interesting is the volatilities are all very similar, meaning that the primary difference in sharpe is due to mean return.

An FX carry trade for MXN and CHF produce positive mean excess return for the sample period, resulting in a positive sharpe ratio. 

## 3.2 Implications for UIP
### 1. Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?
Yes, these stats contradict the log version of UIP. According to UIP, the mean excess returns of these currency positions should be 0, as the change in the spot FX rate is completely explained by changes in the risk-free rates. However, as we see from the stats, none of these currency positions will result in a truly 0 mean excess returns. The closest to 0 is CHF which has a mean excess return of 0.0031 from the FX carry trade.

### 2. A long position in which foreign currency offered the best Sharpe ratio over the sample?
A long position in MXN offers the best Sharpe Ratio of around 0.1 over the sample.

### 3. Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?
Yes, being long on any other currencies apart from MXN and CHF will yield a negative excess return (in USD) over the sample. They all have negative or almost 0 mean excess returns in USD terms, except MXN and CHF.

## 3.3 Predicting FX

In [37]:
# x variable (denoted as factor) =  rf_{USD,t,t+1} - rf_{foreign,t,t+1}
# y variable (denoted as strat) =  spot_{t+1} - spot_{t}
# Annualize by 252 days

k = 'log_JPY'
factor = (risk_free_rates['log_USD'] - risk_free_rates[k])
strat = fx_rates[k].shift(-1) - fx_rates[k]
summ_JPY = calc_multivariate_regression(y=strat, X=factor, intercept=True, adj=252) # Annualize by 252 days

k = 'log_EUR'
factor = (risk_free_rates['log_USD'] - risk_free_rates[k])
strat = fx_rates[k].shift(-1) - fx_rates[k]
summ_EUR = calc_multivariate_regression(y=strat, X=factor, intercept=True, adj=252)

k = 'log_GBP'
factor = (risk_free_rates['log_USD'] - risk_free_rates[k])
strat = fx_rates[k].shift(-1) - fx_rates[k]
summ_GBP = calc_multivariate_regression(y=strat, X=factor, intercept=True, adj=252)

k = 'log_MXN'
factor = (risk_free_rates['log_USD'] - risk_free_rates[k])
strat = fx_rates[k].shift(-1) - fx_rates[k]
summ_MXN = calc_multivariate_regression(y=strat, X=factor, intercept=True, adj=252)

k = 'log_CHF'
factor = (risk_free_rates['log_USD'] - risk_free_rates[k])
strat = fx_rates[k].shift(-1) - fx_rates[k]
summ_CHF = calc_multivariate_regression(y=strat, X=factor, intercept=True, adj=252)

summ_combined = pd.concat([summ_JPY, summ_EUR, summ_GBP, summ_MXN, summ_CHF])
# Show alpha, then beta, then R-Squared. Transpose to switch row and column
summ_combined = summ_combined[['Alpha', '0 Beta', 'R-Squared']].T
summ_combined = summ_combined.rename(index={'0 Beta': 'Beta'})
summ_combined = summ_combined.rename(columns={'log_JPY': 'JPY', 
                                              'log_EUR': 'EUR', 
                                              'log_GBP': 'GBP', 
                                              'log_MXN': 'MXN', 
                                              'log_CHF': 'CHF'})
summ_combined

,JPY,EUR,GBP,MXN,CHF
Alpha,-0.0219,-0.0315,-0.0252,-0.0736,0.0037
Beta,0.3518,2.1602,6.8103,-0.8088,0.9069
R-Squared,0.0000,0.0003,0.0015,0.0000,0.0001


### 3.3.2. Suppose the foreign risk-free rate increases relative to the US rate.

Here we are regressing y (change in FX spot rate (expressed in log USD)) on x (USD interest rate - foreign currency interest rate).

If the spot rate at t+1 is less than the spot rate at t, it means that the USD has strengthened. You need fewer units of USD to purchase 1 unit of foreign currency at time t+1 compared to at time t. 

If the spot rate at t+1 is bigger than the spot rate at t, it means that the USD has weakened. You need more units of USD to purchase 1 unit of foreign currency at time t+1 compared to at time t.

### (a) For which foreign currencies would we predict a relative strengthening of the USD in the following period?
If the risk-free rate of the foreign currency increase relative to the USD risk-free rate, and the beta is positive, then the predicted variable will decrease. If the difference between the spot rate at t+1 and t decrease, it means that the USD has strengthened. 

The ones that have positive beta are JPY, EUR, GBP, and CHF. We are predicting a relative strengthening of the USD in the following period, relative to JPY, EUR, GBP, and CHF. We predict those currencies would have lower FX rates in case the risk-free rate of those currencies increase.

### (b) For which currencies would we predict relative weakening of the USD in the following period?
If the risk-free rate of the foreign currency increase relative to the USD risk-free rate, and the beta is negative, then the predicted variable will increase. If the difference between the spot rate at t+1 and t increase, it means that the USD has weakened. 

The one that has negative beta is MXN. We are predicting a relative weakening of the USD in the following period, relative to MXN. We predict MXN would have higher FX rate in case the risk-free rate of MXN increase.

### (c) This FX predictability is strongest in the case of which foreign currency?
Indicated by the R-Squared in the regression, the FX predictibility seems to be strongest in case of GBP. However, it should be noted that this R-Squared is still fairly low and might not indicate towards a strong enough prediction.

## 3.4 The Dynamic Carry Trade

In [46]:
# Risk free rate USD - risk free rate foreign currency, shift by 1
r_diff = (risk_free_rates['log_USD'] - risk_free_rates['log_JPY']).to_frame('FX Premium').dropna()

alpha = summ_combined.loc['Alpha', 'JPY']/252 # make it daily
beta_adj = summ_combined.loc['Beta', 'JPY'] - 1
expected_fx_premium = alpha + beta_adj*r_diff

positive_premium = len(expected_fx_premium[expected_fx_premium['FX Premium'] > 0]) # number of times it's positive risk premium
total_month = len(expected_fx_premium) # total number of months
positive_premium_percent = positive_premium * 100/len(expected_fx_premium)

# Make a new data frame
positive_table = pd.DataFrame(index = ['JPY'], columns = ['Months - Positive Premium','Total Months','Frequency(%)-Positive Premium'])
positive_table.loc['JPY',:] = positive_premium, total_month, positive_premium_percent
positive_table

,Months - Positive Premium,Total Months,Frequency(%)-Positive Premium
JPY,0,4393,0.0000


In [47]:
# EUR
r_diff = (risk_free_rates['log_USD'] - risk_free_rates['log_EUR']).to_frame('FX Premium').dropna()
alpha = summ_combined.loc['Alpha', 'EUR']/252 # make it daily
beta_adj = summ_combined.loc['Beta', 'EUR'] - 1
expected_fx_premium = alpha + beta_adj*r_diff
positive_premium = len(expected_fx_premium[expected_fx_premium['FX Premium'] > 0]) # number of times it's positive risk premium
positive_premium_percent = positive_premium * 100/len(expected_fx_premium)
positive_table.loc['EUR',:] = positive_premium, total_month, positive_premium_percent

# GBP
r_diff = (risk_free_rates['log_USD'] - risk_free_rates['log_GBP']).to_frame('FX Premium').dropna()
alpha = summ_combined.loc['Alpha', 'GBP']/252 # make it daily
beta_adj = summ_combined.loc['Beta', 'GBP'] - 1
expected_fx_premium = alpha + beta_adj*r_diff
positive_premium = len(expected_fx_premium[expected_fx_premium['FX Premium'] > 0]) # number of times it's positive risk premium
positive_premium_percent = positive_premium * 100/len(expected_fx_premium)
positive_table.loc['GBP',:] = positive_premium, total_month, positive_premium_percent

# MXN
r_diff = (risk_free_rates['log_USD'] - risk_free_rates['log_MXN']).to_frame('FX Premium').dropna()
alpha = summ_combined.loc['Alpha', 'MXN']/252 # make it daily
beta_adj = summ_combined.loc['Beta', 'MXN'] - 1
expected_fx_premium = alpha + beta_adj*r_diff
positive_premium = len(expected_fx_premium[expected_fx_premium['FX Premium'] > 0]) # number of times it's positive risk premium
positive_premium_percent = positive_premium * 100/len(expected_fx_premium)
positive_table.loc['MXN',:] = positive_premium, total_month, positive_premium_percent

# CHF
r_diff = (risk_free_rates['log_USD'] - risk_free_rates['log_CHF']).to_frame('FX Premium').dropna()
alpha = summ_combined.loc['Alpha', 'CHF']/252 # make it daily
beta_adj = summ_combined.loc['Beta', 'CHF'] - 1
expected_fx_premium = alpha + beta_adj*r_diff
positive_premium = len(expected_fx_premium[expected_fx_premium['FX Premium'] > 0]) # number of times it's positive risk premium
positive_premium_percent = positive_premium * 100/len(expected_fx_premium)
positive_table.loc['CHF',:] = positive_premium, total_month, positive_premium_percent

positive_table

,Months - Positive Premium,Total Months,Frequency(%)-Positive Premium
JPY,0,4393,0.0000
EUR,166,4393,3.7787
GBP,1093,4393,24.8805
MXN,3342,4393,76.0756
CHF,4239,4393,96.4944


### 2. Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?
CHF most consistently have a positive FX risk premium. The FX risk premium most often go negative for JPY.

### 3. Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.
Since from 3.1.a, JPY returns seem to be away from the expected value of 0, an improvement in the carry trade would be to short the JPY, i.e. borrow at the JPY risk-free rate to invest in the USD risk-free rate. With our forecast of the USD strengthening relative to the JPY, we could be potentially getting a positive risk premia from this carry trade.